<a href="https://colab.research.google.com/github/lovgon/ML_NIR_2021/blob/main/ML_adv/ML_adv_%D0%A3%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_4_(Python)_%D0%90%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install imutils
!pip install opencv-python
!pip install --upgrade scikit-learn==0.23.0

Подключение базовых библиотек для обработки изображений

In [ ]:
from IPython.display import Image
from imutils import paths
import numpy as np
import cv2
import os

Для работы с изображениями, найдем их гистограммы — характеристики распределения интенсивности изображения. Для этого можно воспользоваться следующим методом.

In [ ]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

Считаем изображения из корневой папки train и вычислим гистограмму каждого изображения. Кроме того, отделим метку каждого изображения. Обработка большого числа изображений в облачных сервисах занимает длительное время (1-5 минут).

In [ ]:
imagePaths = sorted(list(paths.list_images('train')))
trainData = []
labels = []

for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath, 1)
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
    hist = extract_histogram(image)
    trainData.append(hist)
    labels.append(label)

Преобразуем метки в удобный формат 0 и 1. Cat заменяем на 1, Dog на 0.

In [ ]:
Y = [1 if x == 'cat' else 0 for x in labels]

Видно, что классом 1 у нас занумерованы коты (кошки).

Обучаем первый базовый классификатор - бэггинг деревьев принятия решений

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 92)
bagging = BaggingClassifier(tree, n_estimators = 20, random_state = 92)
bagging.fit(trainData, Y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                        max_leaf_nodes=20,
                                                        min_samples_leaf=10,
                                                        random_state=92),
                  n_estimators=20, random_state=92)

Теперь обучим модель почти-разделяющий гиперплосоксти.

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC(C = 1.92, random_state = 92)
svm.fit(trainData, Y)

LinearSVC(C=1.92, random_state=92)

И наконец обучаем третий базовый алгоритм — случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 92)
forest.fit(trainData, Y)

RandomForestClassifier(criterion='entropy', max_leaf_nodes=20,
                       min_samples_leaf=10, n_estimators=20, random_state=92)

Задаем в качестве решающего метаалгоритма логистическую регрессию:

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs', random_state = 92)

Обучаем метаалгоритм

In [ ]:
from sklearn.ensemble import StackingClassifier

base_estimators = [('SVM', svm), ('Bagging DT', bagging), ('DecisionForest', forest)]
sclf = StackingClassifier(estimators=base_estimators, final_estimator=lr, cv = 2)
sclf.fit(trainData, Y)

StackingClassifier(cv=2,
                   estimators=[('SVM', LinearSVC(C=1.92, random_state=92)),
                               ('Bagging DT',
                                BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                        max_leaf_nodes=20,
                                                                                        min_samples_leaf=10,
                                                                                        random_state=92),
                                                  n_estimators=20,
                                                  random_state=92)),
                               ('DecisionForest',
                                RandomForestClassifier(criterion='entropy',
                                                       max_leaf_nodes=20,
                                                       min_samples_leaf=10,
          

Оценка метамодели. Доля правильной классификации (Accuracy):

In [ ]:
'%.2f' % sclf.score(trainData, Y)

'0.85'

Вычислим предсказания для объекта из папки test

In [ ]:
'%.3f' % (sclf.predict_proba(extract_histogram(cv2.imread('test/cat.1046.jpg')).reshape(1, -1))[0][1])

'0.426'

In [ ]:
'%.3f' % (sclf.predict_proba(extract_histogram(cv2.imread('test/dog.1005.jpg')).reshape(1, -1))[0][1])

'0.096'

In [ ]:
'%.3f' % (sclf.predict_proba(extract_histogram(cv2.imread('test/dog.1019.jpg')).reshape(1, -1))[0][1])

'0.271'

In [ ]:
'%.3f' % (sclf.predict_proba(extract_histogram(cv2.imread('test/cat.1001.jpg')).reshape(1, -1))[0][1])

'0.266'